In [ ]:
from obspy.clients.fdsn import Client

client = Client("USGS")
latitude = 43.6150
longitude = -116.2023
maxradius = 5.0 # Based on Angle

# Get earthquake events within a raduis
try:
    catalog = client.get_events(starttime="2020-10-01", endtime="2024-10-11", 
                                minmagnitude=2, latitude=latitude,
                                longitude=longitude, maxradius=maxradius) # Found after checking options for get_events
    
    print(f"Number of 2+ magnitude seismic events near Boise, Idaho: {len(catalog)}")
    
    # Iterate through the events and print details
    for event in catalog:
        temp = event.origins[0]
        location = temp.longitude, temp.latitude
        magnitude = event.magnitudes[0].mag
        event_type = event.event_type
        print(f"Event: {event_type}, Magnitude: {magnitude}")
        print(f"Location: Longitude: {location[0]}, Latitude: {location[1]}")
except Exception as e:
    print(f"An error occurred: {e}")

Event: earthquake, Magnitude: 2.8
Location: Longitude: -115.993, Latitude: 38.618
Event: earthquake, Magnitude: 2.51
Location: Longitude: -112.6565, Latitude: 46.945
Event: earthquake, Magnitude: 3.0
Location: Longitude: -115.0729, Latitude: 44.271
Event: earthquake, Magnitude: 3.0
Location: Longitude: -116.8868, Latitude: 39.5971
Event: earthquake, Magnitude: 2.0
Location: Longitude: -119.0557, Latitude: 42.1055
Event: earthquake, Magnitude: 2.1
Location: Longitude: -115.049833333333, Latitude: 44.2715
Event: earthquake, Magnitude: 2.7
Location: Longitude: -112.4725, Latitude: 45.2249
Event: earthquake, Magnitude: 2.5
Location: Longitude: -112.4657, Latitude: 45.2202
Event: earthquake, Magnitude: 4.1
Location: Longitude: -112.4741, Latitude: 45.228
Event: earthquake, Magnitude: 2.5
Location: Longitude: -119.6872, Latitude: 40.2319
Event: earthquake, Magnitude: 2.22
Location: Longitude: -119.03, Latitude: 42.117
Event: earthquake, Magnitude: 2.13
Location: Longitude: -122.2515, Latitud